In [1]:
import pm4py
pm4py.__version__

'1.3.0'

# Performance Analysis

> Which parts of the process have the biggest influence on the total case duration?

## Data loading

Import the log.

In [33]:
import os
import pandas as pd
from pm4py.objects.conversion.log import factory as log_converter
from pm4py.util import constants
from pm4py.objects.log.importer.xes import importer as xes_importer

PROJ_ROOT = os.path.abspath(os.path.pardir)
data_path = os.path.join(PROJ_ROOT, 'data', 'log.csv')

assert os.path.exists(data_path)

df_log = pd.read_csv(data_path)

# map dataset columns to PM4Py keys
param_keys={constants.PARAMETER_CONSTANT_CASEID_KEY: 'Patient',
            constants.PARAMETER_CONSTANT_RESOURCE_KEY: 'Resource', 
            constants.PARAMETER_CONSTANT_ACTIVITY_KEY: 'Activity',
            constants.PARAMETER_CONSTANT_TIMESTAMP_KEY: 'Timestamp',
            constants.PARAMETER_CONSTANT_START_TIMESTAMP_KEY: 'start_timestamp'}

event_log = log_converter.apply(df_log, parameters=param_keys)

#event_log = xes_importer.apply(PROJ_ROOT+"/data/processed_log.xes")


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use algorithm entrypoint instead


In [23]:
from pm4py.objects.petri.importer import importer as pnml_importer

#import the filtered petri net obtained in Q1
net, initial_marking, final_marking = pnml_importer.apply(PROJ_ROOT+"/results/Q1/filtered_petri.pnml")


---

# a)
> Provide and briefly describe results of your performance analysis. Remember to also consider your current results which may give you a good entry point for a deeper analysis.

In [34]:
print(len(event_log))
from pm4py.statistics.traces.log import case_statistics

print(case_statistics.get_variant_statistics(event_log))


1500
[{'variant': '', 'count': 1500}]


In [29]:
from pm4py.visualization.petrinet import factory as pn_visualizer

from pm4py.algo.discovery.inductive import factory as inductive_miner
#net, initial_marking, final_marking = inductive_miner.apply(event_log, parameters=param_keys)
perf_net_vis = pn_visualizer.apply(net, initial_marking,
                                final_marking=final_marking, variant="frequency",log=event_log)

figures_dir = os.path.join(PROJ_ROOT, 'report', 'figures')

# fix place size
import numpy as np
body = np.array(perf_net_vis.body)
body[body ==  '\tnode [fixedsize=true shape=circle width=0.75]'] = '\tnode [fixedsize=true shape=circle width=1]'
perf_net_vis.body = body

perf_net_vis.render(os.path.join(figures_dir, 'q3_perf_petrinet'),
                 format='pdf',
                 view=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use visualizer module instead.
  


NoConceptNameException: at least an event is without concept:name

---

# b)
> Discuss insights obtained from you analysis, for example identify bottlenecks, and discuss their impact.